In [3]:
import pickle
import numpy as np
import itertools as it
import matplotlib.pylab as plt
import seaborn as sns
from pprint import pprint
from myutils import *
import math
from scipy.stats import kendalltau
import os

%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度

In [4]:
def count_discordant_concordant_pairs(x, y):
    n = len(x)
    discordant_pairs = 0
    concordant_pairs = 0
    equal_pairs = 0
    partial_eq_pairs = 0

    for i in range(n):
        for j in range(i+1, n):
            if (x[i] < x[j] and y[i] > y[j]) or (x[i] > x[j] and y[i] < y[j]):
                discordant_pairs += 1
            elif (x[i] < x[j] and y[i] < y[j]) or (x[i] > x[j] and y[i] > y[j]):
                concordant_pairs += 1
            else:
                # print((x[i], y[i]), (x[j], y[j]))
                if math.isclose(x[i], x[j]) and math.isclose(y[i], y[j]):
                    equal_pairs += 1
                else:
                    partial_eq_pairs += 1
    
    def bucketize(value, bucket_size=1e-9):
        return round(value / bucket_size)
    
    def count_tied_values(arr):
        counts = {}
        for val in arr:
            if bucketize(val) in counts:
                counts[bucketize(val)] += 1
            else:
                counts[bucketize(val)] = 1
        return counts
    
    def calculate_tied_pairs(counts):
        tied_pairs = sum([(count * (count - 1)) // 2 for count in counts.values()])
        return tied_pairs
    
    counts = count_tied_values(x)
    tied_pairs_x = calculate_tied_pairs(counts)
    # print('x_counts:', len(counts), counts)
    # print('tied_pairs_x:', tied_pairs_x)
    
    counts = count_tied_values(y)
    tied_pairs_y = calculate_tied_pairs(counts)
    # print('y_counts:', len(counts), counts)
    # print('y_tied_pairs:', tied_pairs_y)

    return discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, tied_pairs_x, tied_pairs_y, n*(n-1)//2

def calc_tau(discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs):
    tau = (concordant_pairs - discordant_pairs) / math.sqrt((concordant_pairs + discordant_pairs + T_x) * (concordant_pairs + discordant_pairs + T_y))
    return tau

In [5]:
dr_by_event_dir = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/dr_20240417_1452001e84_anchor_by_event"
dr_by_packet_dir = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/dr_20240417_1452001e84_anchor_by_packet"
sr_dir = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/sr"

# BR

In [26]:
route = 'BR'
print(route)
for mm in ['dl_lost', 'dl_excl', 'ul_lost', 'ul_excl']:
    print(f"======= {mm} =======")
    for i, dirpath in enumerate([sr_dir, dr_by_event_dir, dr_by_packet_dir]):
        dir = os.path.join(dirpath, mm, 'results')
        res_file = [os.path.join(dir, s) for s in os.listdir(dir) if s.startswith(route)][0] if i == 0 else [os.path.join(dir, s) for s in os.listdir(dir) if s.startswith(f'{route}_mle')][0]
        # print(res_file)
        with open(res_file, 'rb') as f:
            res = pickle.load(f)
        
        devices = [item[5] for item in res]
        mean_values = [item[1] for item in res]
        ground_truths = [item[3] for item in res]

        # discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs = count_discordant_concordant_pairs(ground_truths, mean_values)
        tau, p_value = kendalltau(ground_truths, mean_values)
        
        if i == 0:
            print('SR -----------', end=' ')
        elif i == 1:
            print('DR By Event --', end=' ')
        else:
            print('DR By Packet -', end=' ')
        print('tau (p-value):', f'{tau} ({p_value})')
    print()

BR
======= dl_lost =======
SR ----------- tau (p-value): 0.7385620915032681 (2.5215800485929758e-06)
DR By Event -- tau (p-value): 0.6463223787167448 (9.68547513977747e-16)
DR By Packet - tau (p-value): 0.6439749608763693 (1.2280011701749441e-15)

======= dl_excl =======
SR ----------- tau (p-value): 0.607843137254902 (0.00022573646929059787)
DR By Event -- tau (p-value): -0.03912363067292644 (0.6268787493474591)
DR By Packet - tau (p-value): -0.1838810641627543 (0.022325371330546795)

======= ul_lost =======
SR ----------- tau (p-value): 0.6209150326797386 (0.0001549082772801415)
DR By Event -- tau (p-value): 0.14871557434344784 (0.06532466502656226)
DR By Packet - tau (p-value): 0.40612300645242355 (4.826716527271656e-07)

======= ul_excl =======
SR ----------- tau (p-value): 0.07189542483660132 (0.7088807630470404)
DR By Event -- tau (p-value): -0.016431924882629106 (0.8382187961733422)
DR By Packet - tau (p-value): -0.20344287949921752 (0.011476386727573505)



# Airport

In [25]:
route = 'A'
print(route)
for mm in ['dl_lost', 'dl_excl', 'ul_lost', 'ul_excl']:
    print(f"======= {mm} =======")
    for i, dirpath in enumerate([sr_dir, dr_by_event_dir, dr_by_packet_dir]):
        dir = os.path.join(dirpath, mm, 'results')
        res_file = [os.path.join(dir, s) for s in os.listdir(dir) if s.startswith(route)][0] if i == 0 else [os.path.join(dir, s) for s in os.listdir(dir) if s.startswith(f'{route}_mle')][0]
        # print(res_file)
        with open(res_file, 'rb') as f:
            res = pickle.load(f)
        
        devices = [item[5] for item in res]
        mean_values = [item[1] for item in res]
        ground_truths = [item[3] for item in res]

        # discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs = count_discordant_concordant_pairs(ground_truths, mean_values)
        tau, p_value = kendalltau(ground_truths, mean_values)
        
        if i == 0:
            print('SR -----------', end=' ')
        elif i == 1:
            print('DR By Event --', end=' ')
        else:
            print('DR By Packet -', end=' ')
        print('tau (p-value):', f'{tau} ({p_value})')
    print()

A
======= dl_lost =======
SR ----------- tau (p-value): 0.8431372549019609 (1.2272689725953054e-08)
DR By Event -- tau (p-value): 0.726277001186339 (2.9122949665757866e-19)
DR By Packet - tau (p-value): 0.7531645569620252 (1.859706232212446e-20)

======= dl_excl =======
SR ----------- tau (p-value): 0.32026143790849676 (0.06849691078121536)
DR By Event -- tau (p-value): 0.04853228962818004 (0.5466358377540781)
DR By Packet - tau (p-value): 0.14529080243235976 (0.07128722698394292)

======= ul_lost =======
SR ----------- tau (p-value): 0.607843137254902 (0.00022573646929059787)
DR By Event -- tau (p-value): 0.3564914065525715 (1.873109443383019e-05)
DR By Packet - tau (p-value): 0.4761516433574152 (1.234560796184773e-08)

======= ul_excl =======
SR ----------- tau (p-value): 0.18954248366013074 (0.293450858549159)
DR By Event -- tau (p-value): 0.37996478911588283 (2.3546615312257084e-06)
DR By Packet - tau (p-value): -0.0293484646587963 (0.7154116278430649)

